In [ ]:
# imports
import re
import time
import pickle
import logging
import gc
import os
import math
import functools
import requests
import random
import glob
import json

import pandas as pd
import numpy as np
import math as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import sklearn

from scipy import stats
from scipy import sparse

from pathlib import Path

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

log = logging.getLogger('log')
log.setLevel(logging.DEBUG)

lhnd = logging.StreamHandler()
lhnd.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
lhnd.setFormatter(formatter)

log.addHandler(lhnd)

%autonotify -a 30

In [ ]:
torch.cuda.is_available()

In [ ]:
mode = 11

In [ ]:
ignore_dumps = False

def lmap(f, arr):
    return list(map(f, arr))

def lfilter(f, arr):
    return list(filter(f, arr))

def foreach(it, f):
    for e in it:
        f(e)
        
def dump(data, name):
    with open('data/' + name, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        
def load(name):
    with open('data/' + name, 'rb') as f:
        return pickle.load(f)
    
def load_or_dump(path, func):
    if not Path('data/' + path).exists() or ignore_dumps:
        res = func()
    
        dump(res, path)
    else:
        res = load(path)
        
    return res


In [ ]:
from time import sleep

with open('auth/token') as f:
    token = f.readline().strip()

def get_info(ids):
    sleep(0.2)
    mc = 'members_count'
    payload = {'v': '5.92', 'access_token': token, 'fields':mc}
    
    str_ids = functools.reduce(
        lambda x, y: x + y,
        lmap(lambda x: str(x) + ',', ids)
    )
    
    print(str_ids)
    
    payload['group_ids'] = str_ids[0:- 1]
    
    r = requests.get('https://api.vk.com/method/groups.getById', 
                     params=payload)
    
    if (not 'response' in r.json()):
        print(r.json())
        
    res = lmap(lambda x: (x['name'], x['screen_name'], "{:,}".format(x[mc]) if mc in x else -1),r.json()['response'])
    
    return(res)

def info_print(lst):
    info = get_info(lst)
    
    print(lmap(lambda x: x[0], info))

In [ ]:
total = 1015925

def raw_data_filter(file):
    # Mapping to events
    res = list()

    i = 0
    
    for line in file:
        cur = line.rstrip().split(',')
        cur = lmap(lambda p: (re.sub(';.*', '', p), re.sub('.*;', '', p)), cur)

        session = list()
        
        for j in range(0, len(cur)):
            try:
                session.append(int(cur[j][1]))
            except ValueError:
                None
                
        res.append(session)

        i = i + 1
                
        if (i % 100000 == 0):
            gc.collect()

            log.debug("%d %% of mapping is done.", i / total * 100)

    
    return res

def raw_data_filter_likes(file):
    # Mapping to events
    res = list()

    i = 0
    
    groups = set()
    
    for line in file:
        cur = line.rstrip().split(',')
        cur = lmap(lambda p: (re.sub(';.*', '', p), re.sub('.*;', '', p)), cur)

        session = set()
        
        for j in range(0, len(cur)):
            try:
                action = int(cur[j][1])
                
                session.add(action)
            
                groups.add(action)
            except ValueError:
                None
                
        res.append((session, None))

        i = i + 1
                
        if (i % 100000 == 0):
            gc.collect()

            log.debug("%d %% of mapping is done.", i / total * 100)

    
    return (res, groups)

In [ ]:
def load_raw(raw, file):
    raw_data = load_or_dump(raw, lambda: raw_data_filter(open(file, "r")))

    log.info("Data loaded")
    
    return raw_data   

def load_comdined(raw_subs, file_subs, raw_likes, file_likes):
    raw_data = load_raw(raw_subs, file_subs)
    
    raw_data_likes = load_raw(raw_likes, file_likes)
    
    log.info("All data loaded")

    raw_data = lmap(lambda x: (x, 0), raw_data)
    
    raw_data_likes = lmap(lambda x: (x, 3), raw_data_likes)
    
    raw_data.extend(raw_data_likes)
    
    raw_data_likes = None
    
    gc.collect()
    
    random.shuffle(raw_data)
    
    return raw_data[:int(total)]

if (mode == 0):
    raw_data = load_raw("raw", "data/public_sessions_2.txt")
    
if (mode == 2):
    total = 10927682
    
    if (ignore_dumps):
        raw_data = load_or_dump('raw_likes', lambda: raw_data_filter_likes(open("data/all_sessions_likes.txt","r")))

        log.info("Data loaded")
        
if (mode == 3):
    raw_data = load_raw('trim_likes', "data/trim_likes.txt")
    
if (mode == 4):
    raw_data = load_comdined("raw", "data/public_sessions_2.txt", "trim_likes", "data/trim_likes.txt")
    
if (mode == 9):
    raw_data = load_raw('raw_small', "data/public_sessions_2_small.txt")
    
if (mode == 10):   
    raw_data = load_raw('raw_likes_small', "data/likes_small.txt")
    
if (mode == 11):
    ignore_dumps = False
    
    total = 185000
    
    raw_data = load_comdined("raw_small", "data/public_sessions_2_small.txt", 
                             "raw_likes_small", "data/likes_small.txt")

In [ ]:
def group_count(data):
    total = dict()

    for i in data:
        for j in i[0]:
            if (j in total.keys()):
                total[j] = total[j] + 1
            else:
                total[j] = 1
                
    return total

In [ ]:
def load_words_data(file):
    words = []
    
    for line in file:
        for word in line.split():
            words.append(word)

    return words

In [ ]:
if (mode == 1):
    words_size = 50000
    
    ignore_dumps = True
    data = load_or_dump('raw_txt', lambda: load_words_data(open("data/text8.txt","r")))
    groups = group_count([[data]])
    
    dictlist = list(groups.items())
    dictlist.sort(key = lambda x: x[1])
    allowed = set(lmap(lambda x: x[0], dictlist[-words_size:]))

    for i in range(len(data)):
        if not data[i] in allowed:
            data[i] = '-1'
            
    groups = group_count([[data]])
    
    data = [[data, []]]
    
    print(len(groups))
    

In [ ]:
min_session_size = 2
max_session_size = 20

def handle_session(session, groups):
    unsub = set()
    sub = set()
    malformed = set()

    for event in session:
        if (event < 0):
            sub_event = -event

            if (sub_event in sub or sub_event in malformed):
                sub.discard(sub_event)
                unsub.discard(sub_event)
                malformed.add(sub_event)
            else:
                unsub.add(sub_event)
        else:
            if (event in unsub or event in malformed):
                unsub.discard(event)
                sub.discard(event)
                malformed.add(event)
            else:
                sub.add(event)

    if (len(sub) >= min_session_size and len(sub) <= max_session_size):
        for event in sub:
            groups.add(event)
        for event in unsub:
            groups.add(event)

        return (sub, unsub)
    
    return None

def initiail_mapping(lst):
    result = []
    groups = set()
    
    for session in lst:
        s = handle_session(session, groups)
    
        if (s != None):
            result.append(s)
        
    return result, groups
    
def initiail_mapping_likes2(lst):
    result = []
    groups = set()
    
    for id, session in enumerate(lst):
        if (id % 100000 == 0):
            log.info("Processing %i", id)
        
        if (len(session) >= min_session_size and 
            len(session) <= max_session_size and 
            len(set(session)) >= min_session_size):
            for event in session:
                groups.add(event)
            
            result.append((session, set()))
    
    return result, groups

def initiail_mapping_combined(lst):
    result = []
    groups = set()
    
    for id, single in enumerate(lst):
        if (id % 100000 == 0):
            log.info("Processing %i", id)
        
        session = single[0]
        
        if (single[1] == 0):
            s = handle_session(session, groups)
            
            if (s != None):
                result.append((s[0], s[1], 0))
        else:
            sub = set(session)

            if (len(sub) >= min_session_size and len(sub) <= max_session_size):
                for event in sub:
                    groups.add(event)

                result.append((sub, set(), single[1]))
    
    return result, groups

def set_map_handle(cnt, min_allowed, session, groups):
    sub = set() 
        
    unsub = None

    for event in session[0]:
        if (cnt[event] > min_allowed):
            sub.add(event)

    if (session[1] != None):
        unsub = set()

        for event in session[1]:
            if (cnt.get(event, -1) > min_allowed):
                unsub.add(event)    

    if (len(sub) >= min_session_size):
        for event in sub:
            groups.add(event)

        if (unsub != None):
            for event in unsub:
                groups.add(event)

        return (sub, unsub)
    
    return None

def set_map(lst, cnt, min_allowed):
    result = []
    groups = set()
    
    for id, session in enumerate(lst):
        if (id % 100000 == 0):
            log.info("Processing %i", id)

        s = set_map_handle(cnt, min_allowed, session, groups)
        
        if (s != None):
            result.append(s)        
    
    return result, groups

def set_map_likes(lst, cnt, min_allowed):
    result = []
    groups = set()
    
    for id, session in enumerate(lst):
        if (id % 100000 == 0):
            log.info("Processing %i", id)

        sub = list() 
        
        unsub = None
        
        for event in session[0]:
            if (cnt[event] > min_allowed):
                sub.append(event)
          
        if (len(sub) >= min_session_size and len(set(sub)) >= min_session_size):
            for event in sub:
                groups.add(event)

            result.append((sub, unsub))
    
    return result, groups

def set_map_combined(lst, cnt, min_allowed):
    result = []
    groups = set()
        
    for id, session in enumerate(lst):
        if (id % 100000 == 0):
            log.info("Processing %i", id)
            
        if (session[2] == 0):
            s = set_map_handle(cnt, min_allowed, session, groups)

            if (s != None):
                result.append((s[0], s[1], session[2]))   
        else:
            sub = list() 
        
            unsub = None

            for event in session[0]:
                if (cnt[event] > min_allowed):
                    sub.append(event)

            if (len(sub) >= min_session_size and len(set(sub)) >= min_session_size):
                for event in sub:
                    groups.add(event)

                result.append((sub, unsub, session[2]))
    
    return result, groups

def drop_uncommon(raw_data, init_mapping_fn, set_map_fn, min_allowed = 10):
    cnt = None
    sorted_cnt = None
    
    data, groups = init_mapping_fn(raw_data)
    cnt = group_count(data) 
    sorted_cnt = sorted(list(cnt.values()))
    
    while (cnt == None or sorted_cnt[0] < min_allowed):
        gc.collect()
        
        data, groups = set_map_fn(data, cnt, min_allowed)
                
        cnt = group_count(data) 
        sorted_cnt = sorted(list(cnt.values()))
        
        log.info("Length of data:   %d", len(data))
        log.info("Total length:     %d", 
                functools.reduce((lambda x, y: x + y), lmap(lambda a: len(a), data))
                )
        log.info("Number of groups: %d", len(groups))
        log.info("Minimum count:    %d\n", sorted_cnt[0])
        
    return data, groups

In [ ]:
if (mode != 2):
    print(raw_data[2])


In [ ]:
ignore_dumps = False

def map_subs(final_data):
    data, groups = load_or_dump(final_data, lambda: drop_uncommon(raw_data, initiail_mapping, set_map, 50))

    most_common = sorted(group_count(data).items(), key = lambda x: x[1], reverse = True)
    
    return data, groups, most_common 

def map_likes(final_data):
    data, groups = load_or_dump(final_data, lambda: drop_uncommon(
        raw_data, 
        initiail_mapping_likes2, 
        set_map_likes, 
        50
    ))

    most_common = sorted(group_count(data).items(), key = lambda x: x[1], reverse = True)
    
    return data, groups, most_common 

def map_combined(final_data):
    data, groups = load_or_dump(final_data, lambda: drop_uncommon(
        raw_data, 
        initiail_mapping_combined, 
        set_map_combined, 
        50
    ))

    most_common = sorted(group_count(data).items(), key=lambda x: x[1], reverse=True) 
    
    return data, groups, most_common  


if (mode == 0):
    ignore_dumps = False
    
    data, groups, most_common = map_subs('final_data')
    
if (mode == 2):
    ignore_dumps = False
    
    data, groups = load_or_dump('final_data_likes', lambda: drop_uncommon(raw_data, lambda x: x, set_map, 50))

    most_common = sorted(group_count(data).items(), key=lambda x: x[1], reverse=True)
    
if (mode == 3):
    ignore_dumps = False
    
    data, groups, most_common = map_likes('final_data_trim_likes')
    
if (mode == 4):
    ignore_dumps = False
    
    data, groups, most_common = map_combined('final_data_combined')
    
if (mode == 9):
    ignore_dumps = False
    
    data, groups, most_common = map_subs('final_data_small')
    
if (mode == 10):
    ignore_dumps = False
    
    data, groups, most_common = map_likes('final_data_small_likes')
    
if (mode == 11):
    ignore_dumps = True
    
    data, groups, most_common = map_combined('final_data_small_combined')

In [ ]:
print(len(groups))

print(len(data))

print(data[0])

In [ ]:
w2i = {w: i for i, w in enumerate(groups)}
i2w = {i: w for i, w in enumerate(groups)}

In [ ]:
fil_path = "data/filtered"

def ds_sub(id, session):
    for sub in session[0]:
        out.write(str(id) + " " + str(w2i[sub]) + "\n")

    for unsub in session[1]:
        out.write(str(id) + " " + str(-w2i[unsub]) + "\n")
        
def ds_sub2(id, session, nl = "\n"):
    for sub in session[0]:
        out.write(str(w2i[sub]) + " ")

    out.write(nl)
    
def ds_like(id, session):
    _s = set(session[0])
                
    for sub in _s:
        out.write(str(id) + " " + str(w2i[sub]) + " " + str(session[0].count(sub)) + "\n")
    
def ds_like2(id, session):
    for sub in session[0]:
        out.write(str(w2i[sub]) + " ")

    out.write("\n")

if not Path(fil_path).exists():
    if (mode == 9):
        with open(fil_path, "w") as out:  
            for id, session in enumerate(data):
                ds_sub(id, session)

        with open("data/filtered_all", "w") as out:  
            for id, session in enumerate(data):
                ds_sub2(id, session)
                
    if (mode == 10):
        with open(fil_path, "w") as out:  
            for id, session in enumerate(data):
                ds_like(id, session)

        with open("data/filtered_all", "w") as out:  
            for id, session in enumerate(data):
                ds_like2(id, session)
                
    if (mode == 11):
        with open(fil_path, "w") as out:  
            for id, session in enumerate(data):
                if (session[2] == 0):
                    ds_sub(id, session)
                else:
                    ds_like(id, session)

        with open("data/filtered_all", "w") as out:  
            for id, session in enumerate(data):
                if (session[2] == 0):
                    ds_sub2(id, session, "*\n")
                else:
                    ds_like2(id, session)

In [ ]:
print(i2w[0])

In [ ]:
raw_data = None

gc.collect()

In [ ]:
session_dex = 0
event_dex = 0

def generate_window_batch(negative_size, window_size = 1):
    assert min_session_size > 1
    
    global session_dex
    global event_dex
    
    labels = []
    batch = None
    negative = np.ndarray(shape=(negative_size), dtype=np.int32)
     
    current = 0
    session = list(data[session_dex][0])
    
    batch = w2i[session[event_dex]]
    
    for j in range(max(0, event_dex - window_size), event_dex):
        labels.append(w2i[session[j]])

    for j in range(event_dex + 1, min(len(session), event_dex + 1 + window_size)):
        labels.append(w2i[session[j]])

    neg = 0

    if (data[session_dex][1] != None):
        for j in data[session_dex]:
            negative[neg] = w2i[j]
            neg += 1
            if (neg == negative_size):
                break

    rand_neg = np.random.randint(len(groups), size=negative_size - neg)

    for j in range(0, negative_size - neg):
        negative[neg + j] = rand_neg[j]

    event_dex += 1

    if (event_dex == len(session)):
        event_dex = 0
        session_dex = session_dex + 1
        if (session_dex >= len(data)):
            session_dex = 0
        session = list(data[session_dex][0])        
     
    return batch, labels, negative


def generate_batch(negative_size):
    assert min_session_size > 1
    
    global session_dex
    global event_dex
    
    session = list(data[session_dex][0])
    
    labels = lmap(lambda x: w2i[x], session)
    batch = w2i[session[event_dex]]
    negative = np.ndarray(shape=(negative_size), dtype=np.int32)
                
    neg = 0

    if (data[session_dex][1] != None):
        for j in data[session_dex][1]:
            negative[neg] = w2i[j]
            neg += 1
            if (neg == negative_size):
                break

    rand_neg = np.random.randint(len(groups), size=negative_size - neg)

    for j in range(0, negative_size - neg):
        negative[neg + j] = rand_neg[j]

    event_dex += 1

    if (event_dex == len(session)):
        event_dex = 0
        session_dex = session_dex + 1
        if (session_dex >= len(data)):
            session_dex = 0 

    return batch, labels, negative

In [ ]:
session_dex = 2

print(data[session_dex])
print(data[session_dex + 1])

if (mode == 3):
    batch, labels, negative = generate_window_batch(10, 2)
else:
    batch, labels, negative = generate_batch(10)
    
print (batch)
print (labels)
print (negative)

print ("\n")

print(i2w[batch], '->', lmap(lambda x: i2w[x], labels), '-> (negative)', lmap(lambda x: i2w[x], negative))
    
print(negative)

In [ ]:
test_ids = lfilter(lambda x: x in w2i, [129440544, 28261334, 92876084, 51016572, 91933860, 22751485])

if (mode == 1):
    test_ids = ['term', 'first', 'used', 'early', 'against', 'working']

In [ ]:
lst = []

for i in data:
    for j in test_ids:
        if j in i[0]:
            f = list(i[0])
            f.remove(j)
            f.append(j)
            lst.append(f)
            
random.shuffle(lst)

if (len(lst) > 0):
    for i in range(10):
        sleep(1)
        print(get_info(lst[i]))
        print()
        print("==================================================")
        print()

In [ ]:
def available_device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device = available_device()

# device = "cpu"

print(device)


In [ ]:
def test(model):
    for i in test_ids:
        t = []
        
        for j in range(vocab_size):
            fst = Variable(torch.LongTensor([w2i[i]]).to(device))
            snd = Variable(torch.LongTensor([j]).to(device))
            t.append([model.score(fst, snd), i2w[j]])
        
        t.sort(key = lambda x: -x[0])
        
        ids = []
        res = t[:10]
       
        for k in res:
            ids.append(k[1])
            
        info = get_info(ids)
        
        print(i)
        for i in range(10):
            print(res[i], ' ', info[i])

In [ ]:
learning_rate = 0.5
vocab_size = len(groups)

window_size = 4
embedding_size = 64
negative_size = 5
batch_size = 1

pref = "/None/"

session_dex = 0
event_dex = 0

In [ ]:
print (data[0])

random.shuffle(data)

print(data[0])

In [ ]:
n_iterations = 501000

def loss_sampled(scores):
    res = scores[0]
    
    for i in range(1, len(scores)):
        res = res + scores[i]
        
    return res * Variable(torch.Tensor([-1]).to(device))

class SkipGram(nn.Module):
    def __init__(self, vocab_size, embd_size):
        super(SkipGram, self).__init__()
        self.in_embeddings = nn.Embedding(vocab_size, embd_size).to(device)
        self.out_embeddings = nn.Embedding(vocab_size, embd_size).to(device)
    
    def forward(self, focus, context):
        embed_focus = self.in_embeddings(focus).view((1, -1))
        embed_ctx = self.out_embeddings(context).view(-1, embedding_size)

        score = torch.mm(embed_focus, torch.t(embed_ctx))
      
        return score
    
    def score(self, focus, context):
        embed_focus = self.in_embeddings(focus).view((1, -1))
        embed_ctx = self.in_embeddings(context).view((1, -1))

        score = F.cosine_similarity(embed_focus, embed_ctx)
    
        return score
    
model = SkipGram(vocab_size, embedding_size).to(device)
    
def train_skipgram():
    losses = []
    loss_fn = loss_sampled
    
    global mode

    print(model)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    total_loss = .0
    
    for i in range(n_iterations):
        lm = mode
        
        c = 1
        
        if (len(data[session_dex]) > 2):
            lm = data[session_dex][2]
            
            if (mode == 0):
                c = 3
        
        if (lm == 3 or lm == 10):
            target, contexts, negative = generate_window_batch(negative_size, window_size)
        else:
            target, contexts, negative = generate_batch(negative_size)
        
        model.zero_grad()

        it_losses = []
        
        scores = []

        in_w_var = Variable(torch.LongTensor([target])).to(device)

        scrs = Variable(torch.LongTensor(contexts)).to(device)
        
        scrs = torch.sigmoid(model(in_w_var, scrs))
        
        scrs = torch.log(scrs)
              
        n_scrs = Variable(torch.LongTensor(negative)).to(device)
        
        n_scrs = torch.sigmoid(model(in_w_var, n_scrs) * Variable(torch.Tensor([-1])).to(device)  )
        
        n_scrs = torch.log(n_scrs)
    
        loss = - (torch.sum(scrs) + torch.sum(n_scrs)) / ((len(contexts) + len(negative)) / c)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        if (i % 2000 == 0 and i > 0):
            if i > 0:
                total_loss /= 2000
                
            if (i % 100000 == 0):
                dump(model, pref + '{0:08d}'.format(i))
#                 test(model)
            
            log.debug('Average loss at step %d: %.4f', i, total_loss)
            total_loss = 0
            losses.append(total_loss)
      
    
    return model, losses

In [ ]:
tr_model, tr_losses = train_skipgram()

# dump(model, pref + "final")

In [ ]:
model = tr_model

In [ ]:
model = load("24-combined/07000000")

In [ ]:
model = load("23-small-likes/00500000")

In [ ]:
test(model)

In [ ]:
def get_all_embed(model):
    return model.in_embeddings(
            Variable(
                torch.LongTensor(
                    list(range(len(groups)))
                )
            ).to(device)
        ).cpu().detach().numpy()

In [ ]:
raw_df = pd.read_csv("data/categories_predict_dataset_v2.csv", index_col = False)
df = raw_df[raw_df.id.isin(w2i)]

In [ ]:
print ("Total groups in main dataset: ", len(groups), " common groups in data sets:", len(df))

In [ ]:
print (len(set(df.general)))
print (len(set(df.detailed)))

In [ ]:
min_cat_count = 200

black_list = []

top = 10

cats = None

if mode < 9:
    cats = {"g":
            ["988a7c0d", "d0157bde", "46988013", "9d802769", "de91548c", 
            "7394d6a2", "1a4668e5", "80d061f1", "8eb49651", "1bd3ff12"],
            "d":
            ['3f496854', '14c24f5a', '36e5c6fd', 'c9b867bc', '75671547', 
             '4262731e', 'ef14d17c', '58d06151', 'dee4c7aa', 'a4e3f544']
           }

if (mode == 9):
    min_cat_count = 40
#     Noisy
    black_list = ["39245b73", "1a4668e5"]
    top = 5

if (mode == 10):
    min_cat_count = 80
    black_list = ["39245b73", "1a4668e5", "1bd3ff12", "dee4c7aa"]
    top = 5
    
if (mode == 11):
    min_cat_count = 0
    black_list = ["39245b73", "1a4668e5", "1bd3ff12", "dee4c7aa"]
    top = 5

def fill_cat_set(t):
    cat_set = set()
    
    cat_dict = dict()

    if (t == "g"):
        cat_lst = list(df.general)
    else:
        cat_lst = list(df.detailed)
        
    for c in cat_lst:
        if c in cat_dict:
            cat_dict[c] += 1
        else:
            cat_dict[c] = 1

    for item in cat_dict.items():
        if (item[1] >= min_cat_count and not item[0] in black_list):
            cat_set.add(item[0])
            
    cat_set = list(cat_set)
    
    if (cats != None):
        cat_set = cats[t]
    
    cat_set = sorted(cat_set, key = lambda x: -cat_dict[x])

    cat_set = set(cat_set[:top])
    
    print(lfilter(lambda x: x[0] in cat_set, cat_dict.items()))
    
    return cat_set, cat_dict

cat_set_g, cat_dict_g = fill_cat_set("g")
cat_set_d, cat_dict_d = fill_cat_set("d")

In [ ]:
from sklearn import svm
from sklearn import linear_model

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

def get_classifiers():
    params = {'verbose': 0, 'n_estimators': 100}

    gbc = GradientBoostingClassifier(**params)
    
    abc = AdaBoostClassifier(n_estimators = 100)
    
    s_clf = svm.LinearSVC()
    
    sgd = linear_model.SGDClassifier(max_iter = 1000, tol = 0.001)
    
    return [gbc, abc, s_clf, sgd]
#     return [gbc]

def f1_class(expected, predicted, classifier):
    f1_micro = f1_score(expected, predicted, average = "weighted")
    
    f1_ = f1_score(expected, predicted, average = None)
    
    f1_ = list(zip(f1_, classifier.classes_))
    
    return {"micro": f1_micro, "classes": f1_}

In [ ]:
times = 5

def classify(all_embed, classifier, t = "g"):
    cat_set = cat_set_g if t == "g" else cat_set_d
    
    cl_data = list(enumerate(all_embed))
    
    df_ids = list(df["id"])

    cl_data = lfilter(lambda x: i2w[x[0]] in df_ids, cl_data)

    if (t == "g"):
        cl_data = lmap(
            lambda x: [x[0], x[1], list(df[df.id == i2w[x[0]]].general)[0]], 
            cl_data
        )
    else:
        cl_data = lmap(
            lambda x: [x[0], x[1], list(df[df.id == i2w[x[0]]].detailed)[0]], 
            cl_data
        )
    
    cl_data = lfilter(lambda x: x[2] in cat_set, cl_data)
    
    cl_train, cl_test = train_test_split(cl_data)
    
    trained = classifier.fit(lmap(lambda x: x[1], cl_train), lmap(lambda x: x[2], cl_train))
    
    f1_res = f1_class(
        lmap(lambda x: x[2], cl_test),
        trained.predict(lmap(lambda x: x[1], cl_test)),
        classifier
    )
    
    return [trained, f1_res]

def classify_model(model_name, classifier):
    return classify(get_all_embed(load(model_name)), classifier)

def to_df(score, t):
    cat_dict = cat_dict_g if t == "g" else cat_dict_d
       
    df = pd.DataFrame({
        "f1_result": lmap(lambda x: format(x[0], ".3f"), score),
        "category": lmap(lambda x: x[1], score),
        "frequency": lmap(lambda x: cat_dict[x[1]], score)
    })
    
    df = df.sort_values(by=["frequency"])
    
    return df

def classify_all(embeds, t):
    classifiers = get_classifiers()
    
    f1_scores = []
    
    for id, c in enumerate(classifiers):
        micro = []
        classes = []
        
        for i in range(times):
            log.info("%d classifier, %d times", id, i)
            
            __, f1 = classify(embeds, c, t)
            
            micro.append(f1["micro"])
                      
            if (len(classes) == 0):
                classes = f1["classes"]
            else:
                for j in range(len(classes)):
                    classes[j] = (classes[j][0] + f1["classes"][j][0], classes[j][1])
        
        log.info("%d classifier is finished", id)
                
        micro = sum(micro) / len(micro)
        
        classes = lmap(lambda x: (x[0] / times, x[1]), classes)
                               
        f1_scores.append((to_df(classes, t), micro))
        

    return f1_scores

In [ ]:
scores = classify_all(get_all_embed(model), "g")

print(scores)

scores = classify_all(get_all_embed(model), "d")

print(scores)

In [ ]:
scores = classify_all(get_all_embed(load("24-combined/02000000")), "g")

print(scores)

scores = classify_all(get_all_embed(load("24-combined/02000000")), "d")

print(scores)

In [ ]:
def map_embeds(embed_dict):   
    res = list(embed_dict.items())
    
    res = lmap(lambda x: (i2w[x[0]], x[1]), res)
    
    res = lmap(lambda x: (w2i[x[0]], x[1]), res)
    
    sorted(res, key = lambda x: x[0])
    
    res = lmap(lambda x: x[1], res)
    
    #return res
    return lmap(lambda x: x / np.linalg.norm(x), res)

In [ ]:
als_embed_file = glob.glob("scala/**/out/" + str(mode) + "/ALS_embeddings/*.json")[0]

als_embeds = []

with open(als_embed_file) as json_file: 
    raw_als_embeds = dict()

    embed_list = []

    for single in json_file.readlines():
        embed_list.append(json.loads(single))

    for emb in embed_list:
        raw_als_embeds[emb['id']] = emb['features']

    als_embeds = map_embeds(raw_als_embeds)

    assert len(als_embeds) == len(groups)


In [ ]:
scores = classify_all(als_embeds, "g")

print(scores)

scores = classify_all(als_embeds, "d")

print(scores)

In [ ]:
lda_embed_file = glob.glob("scala/**/out/" + str(mode) + "/lda_embeddings/part-*")[0]

lda_embeds = []

with open(lda_embed_file) as f: 
    raw_lda_embeds = dict()
    
    embed_list = []
    
    i = 0
    
    for single in f.readlines():
        raw_lda_embeds[i] = lmap(lambda x: float(x),
            lfilter(
                lambda x: len(x), 
                re.split(" ", single.rstrip())
            )
        )
        
        i += 1
            
    lda_embeds = map_embeds(raw_lda_embeds)
    
    assert len(lda_embeds) == len(groups)

In [ ]:
scores = classify_all(lda_embeds, "g")

print(scores)

scores = classify_all(lda_embeds, "d")

print(scores)

In [ ]:
def to_file(df):
    df.to_csv("data/classification_result.csv", index = False)

In [ ]:
def no_embed_classify(t):
    if (t == "g"):
        generals = lmap(lambda x: list(df[df.id == i2w[x]].general), list(range(len(groups))))
    else:
        generals = lmap(lambda x: list(df[df.id == i2w[x]].detailed), list(range(len(groups))))
    
    cat_set = cat_set_g if t == "g" else cat_set_d
    
    missing = []
    dexes = []

    for i in range(len(generals)):
        if (len(generals[i]) == 0 or not (generals[i][0] in cat_set)):
            missing.append(i)
        else:
            dexes.append(i)

    generals = lfilter(lambda x: len(x) != 0 and (x[0] in cat_set), generals)

    generals = lmap(lambda x: x[0], generals)
    
    mdata = np.ndarray(shape=(len(generals), len(data)), dtype=np.int8)

    for i in dexes:
        for ev in data[i][0]:
            g = w2i[ev]

            mdata[g - len(lfilter(lambda x: x <= g, missing))][i] += 1

    assert (len(generals) == len(mdata))
        
    f1_scores = []
        
    for id, c in enumerate(get_classifiers()):
        micro = []
        classes = []
        
        for i in range(times):
            log.info("%d classifier, %d times", id, i)
            
            X_train, X_test, y_train, y_test = train_test_split(mdata, generals)

            X_train = sparse.csr_matrix(X_train)

            X_test = sparse.csr_matrix(X_test)

            trained = c.fit(X_train, y_train)

            pred = trained.predict(sparse.csr_matrix(X_test)) 

            f1 = f1_class(y_test, pred, trained)
            
            micro.append(f1["micro"])
                      
            if (len(classes) == 0):
                classes = f1["classes"]
            else:
                for j in range(len(classes)):
                    classes[j] = (classes[j][0] + f1["classes"][j][0], classes[j][1])
        
        log.info("%d classifier is finished", id)
                
        micro = sum(micro) / len(micro)
        
        classes = lmap(lambda x: (x[0] / times, x[1]), classes)
                               
        f1_scores.append((to_df(classes, t), micro))
        
    return f1_scores

In [ ]:
print(no_embed_classify("g"))

gc.collect()

print(no_embed_classify("d"))

In [ ]:
sess_pred_data = lfilter(
    lambda x: len(x) > 3 and x[len(x) - 1] not in set(x[:(len(x) - 2)]), 
    lmap(lambda x: list(x[0]), data)
)

random.shuffle(sess_pred_data)

In [ ]:
def test_next_predict(embeds, test, top = 50):
    emb_t = lmap(lambda x: Variable(torch.FloatTensor(x)), embeds)
    
    ans = []
    
    in_top = 0
    
    for id, single in enumerate(test):
        if (id % 10 == 0):
            log.info("Step %d", id)
    
        actual = w2i[single[len(single) - 1]]
        
        sum_ = emb_t[w2i[single[0]]]

        for i in range(1, len(single) - 1):
            sum_ = sum_ + emb_t[w2i[single[i]]]
            
        similar = []
                
        for i in range(len(emb_t)):
            similar.append((F.cosine_similarity(emb_t[i].view((1, -1)), sum_.view((1, -1))), i))
            
        similar = sorted(similar, key = lambda x: -x[0])
       
        for i in range(len(similar)):
            if similar[i][1] == actual:
                ans.append(i)
                
                if (i <= top):
                    in_top += 1

        assert (len(ans) == id + 1)
   
    return [sum(ans) / len(ans), in_top / len(ans)] 
            

In [ ]:
test_next_predict(als_embeds, sess_pred_data[0:1000])

In [ ]:
test_next_predict(lda_embeds, sess_pred_data[0:1000])

In [ ]:
test_next_predict(get_all_embed(model), sess_pred_data[0:1000])

In [ ]:
emb_t = lmap(lambda x: Variable(torch.FloatTensor(x)), lda_embeds)

sum_ = emb_t[0]

similar = []

for i in range(len(emb_t)):
    similar.append((F.cosine_similarity(emb_t[i].view((1, -1)), sum_.view((1, -1))), i))
    
similar = sorted(similar, key = lambda x: -x[0])

print(get_info(lmap(lambda x: i2w[x[1]], similar[:10])))

In [ ]:
from sklearn import cluster

num_clusters = 32

all_embed = get_all_embed(model)

km = sklearn.cluster.KMeans(n_clusters = num_clusters)

km.fit(all_embed)

clusters = km.predict(all_embed)

target = clusters[w2i[29534144]]

res = []

for i in range(len(groups)):
    if clusters[i] == target:
        res.append(i2w[i])

if (len(res) < 180):
    print(get_info(res))

In [ ]:
target = clusters[w2i[12648877]]

print (km.cluster_centers_[target])

t = []
        
for j in range(vocab_size):
    cent = Variable(torch.FloatTensor([km.cluster_centers_[target]])).to(device)
    embed_ctx = model.in_embeddings(Variable(torch.LongTensor([j])).to(device)).view((1, -1))

    score = F.cosine_similarity(cent, embed_ctx)
    
    t.append([score, i2w[j]])

t.sort(key = lambda x: -x[0])

ids = []
res = t[:10]

for k in res:
    ids.append(k[1])

info = get_info(ids)

print(i)
for i in range(10):
    print(res[i], ' ', info[i])

In [ ]:
tsne = sklearn.manifold.TSNE(verbose=1)

embed_2d = tsne.fit_transform(all_embed)

In [ ]:
ones = [-1.0 for i in range(embedding_size)]

cls_index = [i for i in range(num_clusters)]

cls_index.sort(key = functools.cmp_to_key(lambda x, y: F.cosine_similarity(
                    torch.FloatTensor(km.cluster_centers_[x]).view((1, -1)), 
                    torch.FloatTensor(km.cluster_centers_[y]).view((1, -1))
              )))

fig, ax = plt.subplots(figsize = (40, 20))

cmap = plt.get_cmap("hsv", num_clusters)

sct = ax.scatter(
    x = lmap(lambda x: x[0], embed_2d), 
    y = lmap(lambda x: x[1], embed_2d), 
    c = lmap(lambda x: cls_index.index(x), predicitons), 
    s = 70,
    cmap = cmap,
    alpha = 0.4
)

plt.show()

In [ ]:
to_show_ids = [138248199, 66322477, 157744447, 166036059, 146079944, 178824336, 130360681, 41523650]
to_show_lables = ["Казахстан",  "Школьные", "Подростковые (Д)", "Подростковые(M)", "Подростковые", "Игры/шутеры", "Ремонт", "K-pop"]

to_show = lmap(lambda x: clusters[w2i[x]], to_show_ids)
to_show.insert(0, None)

cmap = plt.get_cmap("jet", len(to_show) - 1)

cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist = lmap(lambda x: (x[0], x[1], x[2], 0.7), cmaplist)
cmaplist.insert(0, (.5, .5, .5, 0.1))

cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap1', cmaplist, len(cmaplist))

clrs = lmap(lambda x: to_show.index(x) if x in to_show else 0, clusters)

fig, ax = plt.subplots(figsize = (40, 20))

sct = ax.scatter(
    x = lmap(lambda x: x[0], embed_2d), 
    y = lmap(lambda x: x[1], embed_2d), 
    c = clrs, 
    s = 70,
    cmap = cmap
)

hand_list = list(zip(to_show_lables, cmaplist[1:]))

hand_list = lmap(lambda x: mpatches.Patch(color = x[1], label = x[0]), hand_list)

plt.legend(handles = hand_list, prop={'size': 36})

plt.show()

In [ ]:
def zoom_in(x_center, y_center, limit, with_data = False):
    sc_data = zip(
        lmap(lambda x: x[0], embed_2d), 
        lmap(lambda y: y[1], embed_2d), 
        lmap(lambda c: cls_index.index(c), clusters),
        list(range(len(embed_2d)))
    )
      
    sc_data = lfilter(lambda elem: abs(elem[0] - x_center) < limit 
                      and abs(elem[1] - y_center) < limit,
                      sc_data)

    if (len(sc_data) == 0):
        print("No data")
        
        return
    
    fig, ax = plt.subplots(figsize = (10, 5))

    classes = lmap(lambda x: x[2], sc_data)
    
    enum = list(enumerate(set(classes)))
    
    class_remap = dict(lmap(lambda x:(x[1], x[0]), enum))
    classes_map = dict(enum) 
    
    classes = lmap(lambda x: class_remap[x], classes)
    
    cmap = plt.get_cmap("jet", len(set(classes)))
    
    sct = ax.scatter(
        x = lmap(lambda x: x[0], sc_data), 
        y = lmap(lambda x: x[1], sc_data), 
        c = classes, 
        s = 70,
        cmap = cmap
    )
    
    cb = plt.colorbar(sct, spacing = "proportional", ticks = np.linspace(0, len(classes), len(classes) + 1))

    cb.set_alpha(1)
    cb.draw_all()
    
    plt.show()
    
    if (with_data):
        for c in class_remap.keys():
            print(class_remap[c])
            
            sleep(0.5)
            
            info_print(
                lmap(lambda x: i2w[x[3]], lfilter(lambda x: x[2] == c, sc_data))
            )        

In [ ]:
ones = [-1.0 for i in range(embedding_size)]

#cls_types = [None, "9d802769", "e48d7610"]
#cls_lables = [None, "Спорт", "Авто и мото"]
cls_types = [None, "4262731e", "75265a2c"]
cls_lables = [None, "Рецепты и еда", "Стиль, одежда, обувь"]

cls = list(range(len(embed_2d)))

cls = lmap(lambda x: i2w[x], cls)
cls = lmap(lambda x: list(df[df.id == x].detailed), cls)
cls = lmap(lambda x: None if len(x) == 0 else x[0], cls)
cls = lmap(lambda x: 0 if not x in cls_types else cls_types.index(x), cls)

fig, ax = plt.subplots(figsize = (40, 20))

cmap = plt.get_cmap("jet", len(cls_types) - 1)

cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist = lmap(lambda x: (x[0], x[1], x[2], 0.7), cmaplist)
cmaplist.insert(0, (.5, .5, .5, 0.1))

cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, len(cmaplist))

sct = ax.scatter(
    x = lmap(lambda x: x[0], embed_2d), 
    y = lmap(lambda x: x[1], embed_2d), 
    c = cls, 
    s = 70,
    cmap = cmap
)

hand_list = list(zip(cls_lables, cmaplist))

hand_list = lmap(lambda x: mpatches.Patch(color = x[1], label = x[0]), hand_list)

plt.legend(handles = hand_list[1:], prop={'size': 36})

plt.show()